# Hyperwave: knowledge base embedding and setup

In [5]:
import xarray
import os
import numpy as np

In [6]:
# I/O configuration
input_folder = '../data/ncdf'
output_folder = '../data/ncdf'

# Runtime configuration
os.makedirs(output_folder, exist_ok=True)
np.set_printoptions(suppress=True, precision=20)

# Load dataset
dataset = xarray.load_dataarray(os.path.join(input_folder, "aug.dataset.nc"))